## Part 1: Preprocessing

In [90]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [91]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [92]:
attrition_df['Department'].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [93]:
attrition_df['Attrition'].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [94]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]



In [96]:
# Create a list of at least 10 column names to use as X data
columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
           'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
            'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes



Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [97]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [98]:
# Convert your X data to numeric data types however you see fit

# Create a LabelEncoder object
le = LabelEncoder()

# Loop over each column in X_train and X_test
for col in X_train.columns:
    # Only encode columns with object datatype
    if X_train[col].dtype == 'object':
        # Fit the encoder on the training data
        le.fit(X_train[col])
        # Transform both training and testing data
        X_train[col] = le.transform(X_train[col])
        X_test[col] = le.transform(X_test[col])

In [99]:
# Show value counts of 'Yes'/'No' values
X_df['OverTime'].value_counts()


OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [100]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [101]:
# Create a OneHotEncoder for the Department column
ohe = OneHotEncoder()

# Fit the encoder to the training data
ohe.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_encoded_train = ohe.transform(y_train[['Department']])
department_encoded_test = ohe.transform(y_test[['Department']])
department_encoded_train.toarray()






array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [103]:
# Create a OneHotEncoder for the Attrition column
ohe = OneHotEncoder()

# Fit the encoder to the training data
ohe.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_encoded_train = ohe.transform(y_train[['Attrition']])
attrition_encoded_test = ohe.transform(y_test[['Attrition']])
attrition_encoded_train.toarray()


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [104]:
# Find the number of columns in the X training data
X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(10,), name='input')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)


In [106]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)


In [108]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [109]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'categorical_crossentropy'},
              metrics=['accuracy'])

# Summarize the model
model.summary()



Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                            

In [110]:
# Train the model
model.fit(X_train_scaled, {'department_output': department_encoded_train.toarray(),
                           'attrition_output': attrition_encoded_train.toarray()},
          epochs=100, shuffle=True, verbose=2)

Epoch 1/100


35/35 - 1s - loss: 1.3064 - department_output_loss: 0.8240 - attrition_output_loss: 0.4824 - department_output_accuracy: 0.6534 - attrition_output_accuracy: 0.8285 - 740ms/epoch - 21ms/step
Epoch 2/100
35/35 - 0s - loss: 1.1794 - department_output_loss: 0.7718 - attrition_output_loss: 0.4076 - department_output_accuracy: 0.6543 - attrition_output_accuracy: 0.8339 - 52ms/epoch - 1ms/step
Epoch 3/100
35/35 - 0s - loss: 1.1379 - department_output_loss: 0.7622 - attrition_output_loss: 0.3758 - department_output_accuracy: 0.6561 - attrition_output_accuracy: 0.8503 - 49ms/epoch - 1ms/step
Epoch 4/100
35/35 - 0s - loss: 1.1158 - department_output_loss: 0.7541 - attrition_output_loss: 0.3617 - department_output_accuracy: 0.6543 - attrition_output_accuracy: 0.8566 - 59ms/epoch - 2ms/step
Epoch 5/100
35/35 - 0s - loss: 1.0989 - department_output_loss: 0.7475 - attrition_output_loss: 0.3513 - department_output_accuracy: 0.6552 - attrition_output_accuracy: 0.8612 - 53ms/epoch - 2ms/step
Epoch 6/10

In [111]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': department_encoded_test.toarray(),
                                                            'attrition_output': attrition_encoded_test.toarray()},
                                            verbose=2)
test_results                                    

12/12 - 0s - loss: 4.2427 - department_output_loss: 2.7152 - attrition_output_loss: 1.5275 - department_output_accuracy: 0.5163 - attrition_output_accuracy: 0.8342 - 157ms/epoch - 13ms/step


[4.242732048034668,
 2.7152259349823,
 1.5275061130523682,
 0.5163043737411499,
 0.83423912525177]

In [112]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {test_results[3]}")
print(f"Attrition predictions accuracy: {test_results[4]}")


Department predictions accuracy: 0.5163043737411499
Attrition predictions accuracy: 0.83423912525177


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, accuracy is not the best metric to use on this data because the data is imbalanced. For example, regarding Department classification, Reasearch&Development had significantly more samples than the Sales and HR departments which might be biased toward the majority class. Also, with regard to Attrition, the number of employees who left the company is much smaller than the number of employees who stayed. This imbalance can lead to a high accuracy score that is misleading.

2. I chose the 'softmax' activation function for the Department output layer because it is a multi-class classification problem. The 'softmax' function is used to predict the probability of each class, and the sum of all probabilities is equal to 1. For the Attrition output layer, I chose the 'sigmoid' activation function because it is a binary classification problem. The 'sigmoid' function outputs values between 0 and 1, which can be interpreted as probabilities for the two classes.

3. The model can be improved by tuning the hyperparameters, such as the number of hidden layers, the number of neurons in each layer, and the number of epochs. Additionally, we can try different activation functions. Lastly, we can try oversampling or undersampling techniques to address the imbalanced data issue and improve the model's accuracy.
